In [10]:
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
import numpy as np

np.random.seed(34)

In [12]:
def sigmoid(x):

    return np.exp(np.minimum(x, 0))/1+np.exp(-np.abs(x))

In [15]:
#ORのデータセット
x_train_or = np.array([[0, 1], [1, 0], [0, 0], [1, 1]])
y_train_or = np.array([[1], [1], [0], [1]])
x_valid_or, y_valid_or = x_train_or, y_train_or
x_test_or, y_test_or = x_train_or, y_train_or

W_or = np.random.uniform(low=-0.08, high=0.08, size=(2,1)).astype("float32")
b_or = np.zeros(shape=(1,)).astype("float32")

In [2]:
def np_log(x):
    return np.log(np.clip(a=x, a_max=1e+10, a_min=1e-10))

In [3]:
def train_or(x, y, epa = 1.0):

    global W_or, b_or

    batch_size = x.shape[0]

    y_hat = sigmoid(np.matmul(W_or, x)+ b_or)

    cost = -(y*np_log(y_hat) + (1-y)*np_log(1-y_hat)).mean()
    delta = y_hat - y

    